# Step 1 - parse the raw data from Infrabel into usable format

The raw data is coming from the official source: https://infrabel.opendatasoft.com/explore/dataset/stiptheid-gegevens-maandelijksebestanden/information/. 

Use Step 0 notebook to automatically download data using the API.

In [30]:
import json
import numpy as np
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta, time
from tqdm.notebook import tqdm
from copy import deepcopy
from collections import defaultdict
import os
import re

## Technical functions

In [31]:
# Parser of dates from the table
def parse_datetime(dt):
    return datetime.strptime(dt, "%d%b%Y %H:%M:%S")

def parse_date(dt):
    return datetime.strptime(dt, "%d%b%Y")

def get_train_no(row):
    split_relation = row["RELATION"].split(" ")
    return (split_relation[0], int(row["TRAIN_NO"]))

# ---- TIMING FUNCTIONS -------
# If any of the encourtered times are NaN, then it shows that it either is starting
# time or finishing time of a train. If this time is starting time then NaN is arrival time
# If NaN is departure time then this station is the end station. 
# Change NaN values to min date (starting station) and max date (end station)
# for the purpose of sorting the timetable

def is_nan_pd(s):
    if isinstance(s, float):
        if np.isnan(s):
            return True
    return False

def get_planned_dept_time(row):
    if is_nan_pd(row["PLANNED_TIME_DEP"]):
        return datetime.max
    return parse_datetime(row["PLANNED_DATE_DEP"] + " " + row["PLANNED_TIME_DEP"])

def get_real_dept_time(row):
    if is_nan_pd(row["REAL_TIME_DEP"]):
        return datetime.max
    return parse_datetime(row["REAL_DATE_DEP"] + " " + row["REAL_TIME_DEP"])

def get_planned_arrival_time(row):
    if is_nan_pd(row["PLANNED_TIME_ARR"]):
        return datetime.min
    return parse_datetime(row["PLANNED_DATE_ARR"] + " " + row["PLANNED_TIME_ARR"])

def get_real_arrival_time(row):
    if is_nan_pd(row["REAL_TIME_ARR"]):
        return datetime.min
    return parse_datetime(row["REAL_DATE_ARR"] + " " + row["REAL_TIME_ARR"])

# ----  END TIMING FUNCTIONS  ------

def get_station_name(row):
    return row["PTCAR_LG_NM_NL"]

# If delay is NaN then add NaN value to json, otherwise turn it to integer 
def get_arrival_delay(row):
    if is_nan_pd(row["DELAY_ARR"]):
        return np.nan
    return int(row["DELAY_ARR"])

def get_departure_delay(row):
    if is_nan_pd(row["DELAY_DEP"]):
        return np.nan
    return int(row["DELAY_DEP"])

# Datetime converter to str for output to JSON
def datetime_converter(o):
    if isinstance(o, datetime):
        return datetime.strftime(o, "%Y-%m-%d %H:%M:%S")

# Replcae min (max) datetime to None after sorting
def replace_dateminmax(x):
    if (x == datetime.min) or (x == datetime.max) or (x is None):
        return None
    return x

## MAIN CODE - parse the raw data into a set of json strings

Raw data is assumed to come in .csv files of monthly records. The folder given in `base_input_dir` input should contain a list of files with filenames `Data_raw_punctuality_{month}{year}.csv`, where month is a 2-digit and year is a 4-digit number.

Output is written into the folder given in `base_output_dir`.

The function goes through the raw data and decomposes the entries with association to each train. 

**Assumption:** Each train has a unique number (realistic), thus when during one day we encounter entries related to one train will constitute its schedule. 

NaN values in arrival and departure time show whether it is the end station of the train. We change them accrodingly to datetime.datetime class equivalent to be able to sort schedules accrodingly.

Schedules are sorted according to planned departure time. Each file is a json dump of a dict relative to one day of train tracks, keyed by train number.

In [34]:
base_input_dir = Path("./infrabel_raw_data/")
base_output_dir = Path("./infrabel_data_json/")

filename_pattern_str = r"Data_raw_punctuality_(?P<year>\d{4})(?P<month>\d{2}).csv"
filename_pattern = re.compile(filename_pattern_str)
raw_data_files = [f for f in os.listdir(base_input_dir) if f.startswith("Data_raw_punctuality")]

print("Raw data files found:", raw_data_files)

for filename in tqdm(raw_data_files, desc = "Raw files:"):
    # MATCH THE PATTERN
    pattern_match = filename_pattern.match(filename)
    if pattern_match is None:
        continue
    year, month = pattern_match.group("year"), pattern_match.group("month")
    
    # UPLOAD FILE
    file_path = os.path.join(base_input_dir, filename)
    punctuality_pd = pd.read_csv(file_path, header = 0, sep = ",")
    punctuality_pd = punctuality_pd.sort_values(by = ["DATDEP", "TRAIN_NO", 
                                                      "RELATION_DIRECTION", "PLANNED_TIME_DEP"], 
                                                                ascending = [True, True, 
                                                                             True, True])
    punctuality_pd.reset_index(inplace = True, drop = True)
    print("Current file: ", filename, flush = True)

    all_days = sorted(list(punctuality_pd.DATDEP.unique()))
    for cday in tqdm(all_days):
        parsed_cday = parse_date(cday)
        punctuality_one_day_pd = punctuality_pd[punctuality_pd["DATDEP"] == cday]
        punctuality_one_day_pd.reset_index(inplace = True, drop = True)

        day_schedule = defaultdict(list)
        for index, row in punctuality_one_day_pd.iterrows():

            t_no = get_train_no(row)
            station_name = get_station_name(row)

            r_time_arr = get_real_arrival_time(row)
            pl_time_arr = get_planned_arrival_time(row)
            r_time_dep = get_real_dept_time(row)
            pl_time_dep = get_planned_dept_time(row)

            dept_del = get_departure_delay(row)
            arr_del = get_arrival_delay(row)
            day_schedule[t_no].append([station_name, pl_time_arr, r_time_arr, pl_time_dep, 
                                   r_time_dep, arr_del, dept_del])

        out_schedule = {}
        for train, sch in day_schedule.items():
            train = [str(x) for x in train]
            train_name = "".join(train)
            sorted_sch = sorted(deepcopy(sch), key = lambda x: x[3])

            # change datetime.max (min) to None values
            for j, entry in enumerate(sorted_sch):
                for i, s in enumerate(entry[1:5]):
                    sorted_sch[j][i+1] = replace_dateminmax(s)

            out_schedule[train_name] = sorted_sch


        dir_name = datetime.strftime(parsed_cday, "%Y-%m")
        out_dir = os.path.join(base_output_dir, dir_name)
        os.makedirs(out_dir, exist_ok=True)
        file_name = datetime.strftime(parsed_cday, "%Y-%m-%d")
        out_file = os.path.join(out_dir, file_name)

        with open(out_file, "w") as out_f:
            json.dump(out_schedule, out_f, default=datetime_converter)


Raw data files found: ['Data_raw_punctuality_201901.csv']


Raw files::   0%|          | 0/1 [00:00<?, ?it/s]

Current file:  Data_raw_punctuality_201901.csv


  0%|          | 0/31 [00:00<?, ?it/s]